<h1>Sesión N° 02 - Utilizando Operadores de Conjuntos de Datos

<h2>1. Operador UNION

1.1. Conectar con la BD TSQL

In [0]:
USE TSQL;
GO

1.2. El operador UNION ALL devuelve todas las filas de ambas tablas incluyendo duplicados

In [0]:
SELECT country, region, city FROM HR.Employees
UNION ALL -- 100 rows
SELECT country, region, city FROM Sales.Customers;

1.3. El operador UNION ALL devuelve todas las filas de ambas tablas excluyendo duplicados

In [0]:
SELECT country, region, city FROM HR.Employees 
UNION 
SELECT country, region, city FROM Sales.Customers; 

<h2>2. Operador EXCEPT y INTERSECT

2.1. El operador INTERSECT retorna solo las filas encontradas en ambas tablas

In [0]:
SELECT country, region, city FROM HR.Employees
INTERSECT -- 3 distinct rows 
SELECT country, region, city FROM Sales.Customers;

2.2. El operador EXCEPT retorna solo las filas de la tabla tabla derecha (HR.Employees) que no se intersectan con la tabla izquierda.

In [0]:
SELECT country, region, city FROM HR.Employees
EXCEPT 
SELECT country, region, city FROM Sales.Customers;

2.3. El operador EXCEPT retorna solo las filas de la tabla tabla derecha (Sales.Custormers) que no se intersectan con la tabla izquierda.

In [0]:
SELECT country, region, city FROM Sales.Customers
EXCEPT 
SELECT country, region, city FROM HR.Employees;

<h2>3. Utilizando el Operador APPLY

3.1. Crear una función de tabla que acepte una variable SupplierID y retorne el top 3 de los productos más costosos de ese proveedor

In [0]:
CREATE FUNCTION dbo.fn_TopProductsByShipper
(@supplierid int)
RETURNS TABLE
AS
RETURN
	SELECT TOP (3) productid, productname, unitprice
	FROM Production.Products
	WHERE supplierid = @supplierid
	ORDER BY unitprice DESC;
GO

3.2. Probar la función con el proveedor ID=2

In [0]:
SELECT * FROM dbo.fn_TopProductsByShipper(2);


3.3. Probar con el operador CROSS APPLY

In [0]:
SELECT S.supplierid, S.companyname,
	P.productid, P.productname, P.unitprice
FROM Production.Suppliers AS S
CROSS APPLY dbo.fn_TopProductsByShipper(S.supplierid) AS P
ORDER BY S.supplierid ASC, P.unitprice DESC;

3.4. Escribir una consulta para buscar clientes sin ordenes generadas.

In [0]:
SELECT C.custid, C.companyname
FROM Sales.Customers AS C --91 customers
LEFT OUTER JOIN Sales.Orders AS O -- 830 orders
ON C.custid = O.custid --832 results with NULL cust

3.5. Retornar los tres productos más recientemente ordenados por cliente, Utilizando CROSS APPLY

In [0]:
SELECT C.custid, TopOrders.orderid, TopOrders.orderdate
FROM Sales.Customers AS C
CROSS APPLY
(SELECT TOP (3) orderid, orderdate
	FROM Sales.Orders AS O
	WHERE O.custid = C.custid
	ORDER BY orderdate DESC, orderid DESC) AS TopOrders;

3.6. Utilizar OUTER APPLY para incluir a los clientes sin ordenes

In [0]:
SELECT C.custid, TopOrders.orderid, TopOrders.orderdate
FROM Sales.Customers AS C
OUTER APPLY
(SELECT TOP (3) orderid, orderdate
	FROM Sales.Orders AS O
	WHERE O.custid = C.custid
	ORDER BY orderdate DESC, orderid DESC) AS TopOrders; 